In [103]:
import pandas as pd
import numpy as np
Sbase=100
def createMatrixX(lines,NB):
    X=np.zeros((NB,NB))
    for i in range(len(lines)):
        nodo_i=int(lines[i,0])-1
        nodo_j=int(lines[i,1])-1
        X[nodo_i,nodo_j]=lines[i,3]
        X[nodo_j,nodo_i]=lines[i,3]
    return X

def createMatrixB(X):
    B=X.copy()
    for i in range(len(B)):
        for j in range(len(B)):
            if i==j:
                B[i,i]=sum(1/k for k in X[i,:] if k!=0)
            else:
                if X[i,j]!=0:
                    B[i,j]=-1/X[i,j]
                else:
                    B[i,j]=0
    return B

def fillPd(loads,NB):
    Pd=[0]*NB
    for i in range(NB):
        ii=int(loads[i,0])
        Pd[ii-1]=loads[i,1]/Sbase
    return Pd

    
def initCosts(generators):
    NG=generators.shape[0]
    Alpha = [0]*NG
    Beta = [0]*NG
    Gamma = [0]*NG
    for i in range(NG):
        Alpha[i]=generators[i,3]
        Beta[i]=generators[i,4]
        Gamma[i]=generators[i,5]
    return Alpha, Beta, Gamma
    
def initPmaxmin(generators):
    NG=generators.shape[0]
    Pmax=[0]*NG
    Pmin=[0]*NG
    for i in range(NG):
        ii=int(generators[i,0])
        Pmin[ii-1]=generators[i,6]/Sbase
        Pmax[ii-1]=generators[i,7]/Sbase
    return Pmin,Pmax

def cost_der(x):
    g=np.zeros_like(x)
    n=len(Beta)
    for i in range(n):
        g[i]=Beta[i]
    g[n:]=0
    return g

def cost_hess(x):
    h=np.zeros((len(x),len(x)))
    return h

def makeConstraint(NB,NL,NG,slack,file,X,B,Pd,Cap):
    dflines=pd.read_excel(file,'ParametrosLineas')
    lines=dflines.values
    binf=np.zeros(NB+NL)
    bsup=np.zeros(NB+NL)
    A=np.zeros((NB+NL,NG+NB))
    for i in range(NB):
        A[i,i]=1
        for j in range(NG,NG+NB):
            A[i,j]=B[i,j-NG]
    #Constraints of flows (angles)
    bsup[0:NB]=np.copy(Pd)
    binf[0:NB]=np.copy(Pd)
    k=0
    for line in lines:
        i=int(line[0])-1
        j=int(line[1])-1
        A[NB+k,NB+i]=1
        A[NB+k,NB+j]=-1
        bsup[NB+k]=Cap*X[i,j]
        binf[NB+k]=-Cap*X[i,j]
        k=k+1
    #Take out slack angle (because theta0 = 0)
    aux=[i for i in range(NG+NB) if i!=(NG+slack)]
    return A[:,aux],binf,bsup
    
def readsystem(file):
    dflines=pd.read_excel(file,'ParametrosLineas')
    lines=dflines.values
    dfgenerators=pd.read_excel(file,'ParametrosGeneradores')
    generators=dfgenerators.values
    dfloads=pd.read_excel(file,'ParametrosCargas')
    loads=dfloads.values
    dfbus=pd.read_excel(file,'ParametrosNodos')
    bus=dfbus.values
    #Number of busbar
    NB=bus[0,0]
    #Number og generators
    NG=generators.shape[0]
    #Number of lines
    NL=len(lines)
    #En pu
    X=createMatrixX(lines,NB)
    B=createMatrixB(X)
    #Pmax, Pmin
    Pmin,Pmax=initPmaxmin(generators)
    #Vector de potencia demandada
    Pd=fillPd(loads,NB)
    #Costos
    Alpha, Beta, Gamma=initCosts(generators)
    return NB,NL,NG,Pd, Alpha, Beta, Gamma,X,B,Pmin,Pmax

def makeBounds():
    lb=[0]*(NG+NB-1) #slack angle=0
    ub=[0]*(NG+NB-1)
    for i in range(NG):
        lb[i]=Pmin[i]
        ub[i]=Pmax[i]
    for i in range(NG,NG+NB-1):
        lb[i]=-1
        ub[i]=1
    bounds = Bounds(lb,ub)
    return bounds

def printFlows(res):
    print('Flows [pu]')
    Theta=res.x[NB:]
    Theta=np.insert(Theta,0,0,axis=0)
    for i in range(1,len(Theta)):
        Thetai=Theta[i-1]
        Thetaj=Theta[i]
        Fij=-(Thetai-Thetaj)/X[i-1,i]
        print('P'+str(i-1)+str(i)+' = ',round(Fij,2))

def printBalance(res):
    print('Balance [pu]')
    print('Generation = ',sum(res.x[0:NG]))
    print('Demand = ',sum(Pd))
    
def printGeneration(res):
    print('Generation by Units [pu]')
    G=res.x[0:NG]
    for i in range(NG):
        print('PG'+str(i)+' = ',round(G[i],2))
        
def printAngles(res):
    print('Angles [pu]')
    Theta=res.x[NB:]
    Theta=np.insert(Theta,0,0,axis=0)
    for i in range(len(Theta)):
        print('Theta'+str(i)+' = ',round(Theta[i],4))
        
def printCosts(res):
    print('System Costs [USD]')
    print('Cost = ',round(np.dot(res.x[0:NG],Beta),2))
    
def makeStartPoint():
    y0=np.zeros(NG+NB-1)
    for i in range(NG):
        y0[i]=0.1
    for i in range(NG,NG+NB-1):
        y0[i]=0
    return y0

# Import Data

In [104]:
import pandas as pd
import numpy as np
from __future__ import print_function
from ortools.linear_solver import pywraplp

#Cargar datos
file='Data_radial_2.xlsx'
NB,NL,NG,Pd, Alpha, Beta, Gamma,X,B,Pmin,Pmax = readsystem(file)

# Solution

In [106]:
from scipy.optimize import Bounds,LinearConstraint, minimize

bounds = makeBounds()

slack=0
Cap=1 #Use Cap=0.0001 to reduce flows to 0

#from Ax=b, A matrix
A,binf,bsup=makeConstraint(NB,NL,NG,slack,file,X,B,Pd,Cap)
linear_constraint = LinearConstraint(A,binf,bsup)         


def fo(y):
    return np.dot(y[0:NG],Beta)

y0 = makeStartPoint()
res = minimize(fo, y0, method='trust-constr',jac=cost_der,
               hess=cost_hess, constraints=linear_constraint,
               options={'verbose': 1,'maxiter':1024}, bounds=bounds)


#PRINT RESULTS
print('--------------------------')
printBalance(res)
print('--------------------------')
printGeneration(res)
print('--------------------------')
printFlows(res)
print('--------------------------')
printAngles(res)
print('--------------------------')
printCosts(res)

`gtol` termination condition is satisfied.
Number of iterations: 8, function evaluations: 6, CG iterations: 5, optimality: 2.34e-12, constraint violation: 5.55e-17, execution time: 0.018 s.
--------------------------
Balance [pu]
Generation =  1.1
Demand =  1.1
--------------------------
Generation by Units [pu]
PG0 =  0.6
PG1 =  0.5
--------------------------
Flows [pu]
P01 =  0.3
--------------------------
Angles [pu]
Theta0 =  0.0
Theta1 =  0.03
--------------------------
System Costs [USD]
Cost =  69.0
